# Bounding configuration GP example

This notebook is designed to test whether a simple simple comprised of bounding boxes can be generated using [deap](http://deap.readthedocs.io/en/master/api/tools.html), a Python Evolutionary Algorithm Package.

In [1]:
import copy

import numpy as np

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp


In [2]:
# def progn(*args):
#     for arg in args:
#         arg()

# def prog2(out1, out2): 
#     return partial(progn,out1,out2)

# def prog3(out1, out2, out3):     
#     return partial(progn,out1,out2,out3)

# def if_then_else(condition, out1, out2):
#     out1() if condition() else out2()



In [69]:
from OCC.gp import gp_Pnt
from airconics.base import AirconicsShape, AirconicsCollection
from airconics import LiftingSurface, Fuselage, Engine
from OCC.gp import gp_Ax2
import OCC
from airconics import AirCONICStools as act
from OCC.BRepPrimAPI import BRepPrimAPI_MakeBox
import pydot


class TreeNode(object):
    def __init__(self, part, name, arity):
        """Basic type to define node elements in the topology tree. To be used
        by Topology class.

        Parameters
        ----------
        part - Airconics type Fuselage, LiftingSurface or Engine
            The type to convert

        name - string
            The name of the part (e.g. 'Wing' or 'Fin')

        arity - int
            the number of descendants

        Attributes
        ----------
        arity - int
            arity (number of descendants) of this node.

        name - string
            Name of the part

        func - string
            Indicates the type of node i.e.
        """
        self.name = name
        self.arity = arity

        if type(part) not in FUNCTIONS.values():
            raise TypeError("Not a recognised part type: {}. Should be {}"
                .format(type(part), FUNCTIONS.values()))
        else:
            func_str = FUNCTIONS_INV[type(part)]
        self.func = func_str

    def __str__(self):
        output = '({}, {}, {})'.format(self.name, self.func, self.arity)
        return output


# Create a simple Box class that inherits from AirconicsShape
#  - this will behave similarly to the Engine, Fuselage class etc.
class Box(AirconicsShape):
    def __init__(self, xmin, ymin, zmin, dx, dy, dz):
        # This implicitly calls build
        super(Box, self).__init__(construct_geometry=True,
                                             xmin=xmin,
                                             ymin=ymin,
                                             zmin=zmin,
                                             dx=dx,
                                             dy=dy,
                                             dz=dz)
        
        
        xmin, ymin, zmin, xmax, ymax, zmax = self.Extents()
        self.xmin = xmin
        self.ymin = ymin
        self.zmin = zmin
        self.xmax = xmax
        self.ymax = ymax
        self.zmax = zmax
        
        directions = []
        
    def Build(self):
        Xmin = gp_Pnt(self.xmin, self.ymin, self.zmin)
        Xmax = gp_Pnt(self.xmin + self.dx, self.ymin + self.dy, self.zmin + self.dz)
        self['Box'] = BRepPrimAPI_MakeBox(Xmin, Xmax).Shape()

        
# This dictionary will be used for topology tree formatting
FUNCTIONS = {'E': Fuselage,         # E = Enclosure
             'L': LiftingSurface,   # L = Lifting Surface
             'P': Engine,           # P = Propulsion
             '|': gp_Ax2,           # M = Mirror Plane
             'B': Box,
             '': None}

# Reversed dictionary for manually adding shapes, i.e. converting
#  a class instance to a string
FUNCTIONS_INV = {func: name for name, func in FUNCTIONS.items()}

# The shapes of nodes in the exported graph from Topo class:
SHAPES = {'E': 'ellipse',
          'L': 'box',
          'P': 'hexagon',
          '|': '',
          'B': 'box',
          '' : 'point'}    
        
class BoxLayout(AirconicsCollection):
    def __init__(self):
        """maxcompoents is the number of components"""
        super(BoxLayout, self).__init__(parts={}, construct_geometry=True)
        
        # Carry around the tree for visualisation purposes:
        self._Tree = None
        
        # Start with a simple box
        self.nparts = 0
        self.routine = None
    
    
    def __setitem__(self, name, part_w_arity):
        """Overloads the assignment operator used by AirconicsCollection
        to allow only tuples as inputs - arity must be specified for
        topology.

        Parameters
        ----------
        name - string
        part_w_arity - tuple
            (Airconics class, int), eg: (Fuselage, 2) is a Fuselage shape with
            2 descendents in its topological tree

        Notes
        -----
        appends to the self.Tree and self._OrderedParts attributes
        """
        try:
            part, arity = part_w_arity
        except:
            print("Warning: no arity set. Treating as zero")
            part = part_w_arity
            arity = 0

        node = TreeNode(part, name, arity)

        self._Tree.append(node)
        super(BoxLayout, self).__setitem__(name, part)
    
    def run(self, tree, pset):
        self._reset()
        routine = gp.compile(tree, pset)
        self._Tree = tree
        routine()

    def _reset(self):
        self._Parts = {}
        self._Tree = None
        self.nparts = 0

    def boxN(self, xmin, ymin, zmin, dx, dy, dz, *args):
        # Fits N new components to this box layout
        box = Box(xmin, ymin, zmin, dx, dy, dz)
        self.nparts += 1
        # Do no be confused between the numbering of boxes and the number of descendent
        #  nodes: Each box needs a unique ID, so naming a box0 function "box0" replaces
        #  other shapes in this layout that are also named box0
        self['box' + str(self.nparts)] = box, len(args)
        for arg in args:
            arg()

    # Now we need to explicitly add the box1, box2, .... boxM functions,
    # So that they can be added to the GP toolbox
    def box0(self, xmin, ymin, zmin, dx, dy, dz):
        return partial(self.boxN, xmin, ymin, zmin, dx, dy, dz)
        
    def box1(self, xmin, ymin, zmin, dx, dy, dz, out1):
        return partial(self.boxN, xmin, ymin, zmin, dx, dy, dz, out1)

    def box2(self, xmin, ymin, zmin, dx, dy, dz, out1, out2):
        return partial(self.boxN, xmin, ymin, zmin, dx, dy, dz, out1, out2)
    
    def temp_fitness(self):
        """Until I come with something useful, the fitness evaluation for this
        configuration will be based on the bounding box volume"""
        try:
            xmin, ymin, zmin, xmax, ymax, zmax = self.Extents()
        except:
            # Bounding Box was probably void
            return 0
        return (xmax-xmin) * (ymax-ymin)*(zmax-zmin)
    
    def export_graphviz(self):
        """Returns a human readable interpretation of the topology tree"""
        # Note: ns below is a simple range list for every edge/label
        ns, edges, labels = gp.graph(self._Tree)
        

        graph = pydot.Dot(type='digraph')

        cluster_1 = pydot.Cluster('standard', label='standard')
        for label in labels:
            pydot_node = pydot.Node(label, shape='box')
            cluster_1.add_node(pydot_node)

        if 
        cluster_2 = pydot.Cluster('mirrored', label='mirrored')


# print(config.export_

#     def export_graphviz(self):
#         """Returns a string, Graphviz script for visualizing the topology tree.

#         Currently only set up to allow a single mirror terminal

#         Returns
#         -------
#         output : string
#             The Graphviz script to plot the tree representation of the program.

#         Notes
#         -----
#         This function is originally from GPLearns _Program class, but has been
#         modified. Can be visualised with pydot,

#         :Example:
#             >>> topo = Topology()     # Add some parts with topo.addPart
#             >>> graph = pydot.graph_from_dot_data(topo.export_graphviz())
#             >>> Image(graph.create_png())

#         May add a dependency on GPLearn later and overload the appropriate
#         class methods.
#         """
#         terminals = []
#         # Assume the geometry is not mirrored at first
#         mirror_flag = False
#         output = """digraph program {
#         splines=ortho;
#         ranksep="0.1";
#         node [style=filled]
#         edge [arrowhead=none];\n"""
        
#         # check if a mirror node is present, in which case, add two subclusters
#         types = [node.func for node in self._Tree]

#         if FUNCTIONS_INV[gp_Ax2] in types:
#             mirror_flag = True
#             output += """# Start the primary cluster (assume only one mirror exists, leading
#             # to two clusters)
#             subgraph cluster_0 {
#                 color=invis;\n"""

#         for i, node in enumerate(self._Tree):
#             if node.func == '|':
#                 # Mirror node branch
#                 output += '}\n'     # Close the primary cluster
#                 output += 'subgraph cluster_1{\nstyle=dashed\n'

#             else:
#                 # All other functions, e.g., Enclosure, LiftingSurface,
#                 # Propulsion
#                 fill = "#136ed4"
#                 if node.arity > 0:
#                     terminals.append([node.arity, i])
#                     output += ('%d [label="%s", fillcolor="%s", shape="%s"] ;\n'
#                                % (i, node.name, fill, SHAPES[node.func]))
#                     # Add a point below to allow orthogonal branching in graphs
#                     output += ('p%d [shape=point, color=none];\n' %(i))
#                     output += ('%d -> p%d;\n' %(i, i))
#                 else:
#                     output += ('%d [label="%s", fillcolor="%s", shape="%s"] ;\n'
#                                % (i, node.name, fill, SHAPES[node.func]))
#                     if i == 0:
#                         # A degenerative program of only one node
#                         return output + "}"
#                     terminals[-1][0] -= 1
#                     terminals[-1].append(i)
#                     while terminals[-1][0] == 0:
#                         output += 'p%d -> %d ;\n' % (terminals[-1][1],
#                                                      terminals[-1][-1])
#                         terminals[-1].pop()
#                         if len(terminals[-1]) == 2:
#                             parent = terminals[-1][-1]
#                             terminals.pop()
#                             if len(terminals) == 0:
#                                 if mirror_flag:
#                                     # close current cluster and end digraph
#                                     return output + "}\n}"
#                                 else:
#                                     return output + "}"

#                             terminals[-1].append(parent)
#                             terminals[-1][0] -= 1

#         # We should never get here
#         return output


In [70]:
config = BoxLayout()
import types, itertools, functools
funct = types.FunctionType
part = functools.partial
nt = types.NoneType

# I think the second argument means that we have up to 57 floats to use as terminals
pset = gp.PrimitiveSetTyped("MAIN", [], nt)


pset.addPrimitive(config.box0, [float, float, float, float, float, float], nt)
pset.addPrimitive(config.box1, [float, float, float, float, float, float, nt], nt)
pset.addPrimitive(config.box2, [float, float, float, float, float, float, nt, nt], nt)
pset.addPrimitive(np.random.rand, [], float)

def useless_None():
    return None
pset.addTerminal(useless_None, nt)

pset.addEphemeralConstant('rand', np.random.rand, float)


creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("expr_init", gp.genFull, pset=pset, min_=1, max_=4)

# Structure initializers
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr_init)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


def evalTopology(individual):
    # Transform the tree expression to functionnal Python code
    routine = gp.compile(individual, pset)
    # Run the generated routine
    global config
    config.run(routine)
    return config.temp_fitness(),
    
toolbox.register("evaluate", evalTopology)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

def main():
    np.random.seed(69)
    
    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    algorithms.eaSimple(pop, toolbox, 0.5, 0.2, 40, stats, halloffame=hof)
    
    return pop, hof, stats


Attempting to construct BoxLayout geometry...


In [72]:
# Try to compile a single individual and visualise both the tree and geometry
tree = toolbox.individual()
print(tree)


config.run(tree, pset)
print(config)


from airconics.Addons.WebServer.TornadoWeb import TornadoWebRenderer

renderer = TornadoWebRenderer()

config.Display(renderer)

renderer

box0(0.05512091369177419, 0.16861109075891834, 0.6436862419933165, 0.06509488490676674, 0.5682016997293703, 0.12211572800784809)
['box1']


<TornadoWebRenderer at http://PChambers_ngcm:34336/get/f479a5dccf2b48caaf6c83a1b6d3615c>

In [90]:
# Visualise the current tree:
from airconics import Topology
from IPython.display import Image

nodes, edges, labels = gp.graph(tree)

for edge in edges:
    print(edge)

    
print(labels)

# graph = pydot.Dot(type='digraph')

# cluster_1 = pydot.Cluster('standard', label='standard')
# for node in nodes:
#     pydot_node = pydot.Node(label, shape='box')
#     cluster_1.add_node()

# cluster_2 = pydot.Cluster('mirrored', label='mirrored')


# print(config.export_graphviz())

# graph = pydot.graph_from_dot_data(config.export_graphviz())
# Image(graph.create_png())

(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
{0: 'box0', 1: 0.05512091369177419, 2: 0.16861109075891834, 3: 0.6436862419933165, 4: 0.06509488490676674, 5: 0.5682016997293703, 6: 0.12211572800784809}


In [92]:
help(pydot.Edge)

Help on class Edge in module pydot:

class Edge(__builtin__.object, Common)
 |  A graph edge.
 |  
 |  This class represents a graph's edge with all its attributes.
 |  
 |  edge(src, dst, attribute=value, ...)
 |  
 |  src: source node's name
 |  dst: destination node's name
 |  
 |  All the attributes defined in the Graphviz dot language should
 |  be supported.
 |  
 |      Attributes can be set through the dynamically generated methods:
 |  
 |   set_[attribute name], i.e. set_label, set_fontname
 |  
 |  or directly by using the instance's special dictionary:
 |  
 |   Edge.obj_dict['attributes'][attribute name], i.e.
 |  
 |      edge_instance.obj_dict['attributes']['label']
 |      edge_instance.obj_dict['attributes']['fontname']
 |  
 |  Method resolution order:
 |      Edge
 |      __builtin__.object
 |      Common
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, edge)
 |      Compare two edges.
 |      
 |      If the parent graph is directed, arcs linking
 |      node A 

In [47]:
pop, hof, stats = main()

hof

gen	nevals	avg     	std    	min       	max    
0  	100   	0.805104	1.06347	0.00025539	5.09885
1  	65    	2.48688 	1.10807	0.0530288 	5.09885
2  	58    	3.59646 	0.987051	0.528565  	5.09885
3  	68    	3.81241 	1.01278 	0.99511   	5.22333
4  	66    	3.92395 	1.03401 	1.41299   	6.33626
5  	59    	3.92606 	1.19715 	1.44178   	6.33626
6  	52    	4.28459 	1.14467 	1.99517   	6.33626
7  	70    	4.44454 	1.21143 	1.91363   	6.33626
8  	61    	5.00241 	1.22087 	1.49155   	7.10264
9  	47    	5.46723 	1.12518 	2.03664   	7.10264
10 	72    	5.06036 	1.23231 	1.30968   	7.10264
11 	55    	5.3884  	1.21447 	1.80115   	7.10264
12 	56    	5.44134 	1.26775 	2.5267    	7.10264
13 	67    	5.15684 	1.45315 	1.6442    	7.10264
14 	66    	5.1475  	1.51794 	2.11211   	7.10264
15 	71    	5.00423 	1.45831 	2.11922   	7.10264
16 	80    	4.47232 	1.28649 	1.49707   	7.10264
17 	62    	5.0085  	1.36937 	2.34188   	7.10264
18 	58    	5.35099 	1.5204  	0.354827  	7.10264
19 	58    	5.27926 	1.70036 	1.65782   	7.1

In [7]:
best = hof[0]

nodes, edges, labels = gp.graph(best)
print(best)

routine = gp.compile(best, pset)
config.run(routine)

print(config)

from airconics.Addons.WebServer.TornadoWeb import TornadoWebRenderer

renderer = TornadoWebRenderer()

config.Display(renderer)
renderer

box2(rand(), rand(), rand(), rand(), rand(), rand(), box2(rand(), rand(), rand(), rand(), rand(), rand(), box2(rand(), rand(), rand(), 0.7036617917548482, rand(), rand(), box2(0.8748633748188587, 0.5595595170406705, 0.9170047415837121, 0.4423162143316315, 0.3206571462359713, 0.19464673114229025, useless_None, useless_None), box2(0.49707731271128075, 0.9062991224568844, 0.5690672037124594, 0.359820411774053, 0.4956880350756393, 0.4119184433609153, useless_None, useless_None)), box0(rand(), rand(), rand(), rand(), rand(), rand())), box2(rand(), rand(), rand(), rand(), rand(), rand(), box2(rand(), rand(), rand(), rand(), rand(), rand(), box0(0.4958887344990667, 0.4241292773949338, 0.8551895290463735, 0.49426737813893495, 0.25841518147387266, 0.23782133564114438), box2(0.17956034310096147, rand(), 0.578854630578671, 0.41733522546876556, 0.8151192627896259, 0.031054394032625998, useless_None, useless_None)), box2(rand(), rand(), rand(), rand(), rand(), rand(), box1(0.5791585008689591, 0.403

<TornadoWebRenderer at http://PChambers_ngcm:45262/get/f16f1066a47d45c7944a1abb2127e001>

In [ ]:
# Visualise the best tree:
